In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of Bollywood 2022 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_Hindi_films_of_2022"

In [21]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [22]:
tables = soup.find_all('table',class_='wikitable')

In [23]:
len(tables)

6

In [24]:
type(tables[0])

bs4.element.Tag

In [40]:
df1 = pd.read_html(str(tables[2]))[0]
df2 = pd.read_html(str(tables[3]))[0]
df3 = pd.read_html(str(tables[4]))[0]
df4 = pd.read_html(str(tables[5]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [42]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_12272\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [43]:
df

,Opening,Opening.1,Title,Director,Cast,Studio (production house),Ref.
0,J A N,21.0,36 Farmhouse,Ram Ramesh Sharm,Vijay RaazSanjay MishraAmol ParasharBarkha Sin...,"Mukta Arts, ZEE5",[16]
1,J A N,26.0,Hai Tujhe Salaam India,Avanish Kumar,Aarya BabbarAjaz KhanSmita GondkarKanwalpreet ...,Redwood Productions,[17]
2,F E B,4.0,Looop Lapeta,Aakash Bhatia,Taapsee PannuTahir Raj Bhasin,"Sony Pictures Films India, Ellipsis Entertainm...",[18]
3,F E B,11.0,Gehraiyaan,Shakun Batra,Deepika PadukoneSiddhant ChaturvediAnanya Pand...,"Viacom18 Studios, Dharma Productions, Jouska F...",[19]
4,F E B,11.0,Badhaai Do,Harshavardhan Kulkarni,Rajkummar RaoBhumi Pednekar,"Junglee Pictures, Zee Studios",[20]
...,...,...,...,...,...,...,...
100,D E C,16.0,No Means No,Vikash Verma,Dhruv VermaGulshan GroverSylwia CzechSharad Ka...,G7 Films Poland,[113]
101,D E C,17.0,The Good Maharaja,Vikash Verma,Sanjay DuttDhruv VermaSharad KapoorGulshan Grover,G7 Films Poland,[114][115][116]
102,D E C,23.0,Cirkus,Rohit Shetty,Ranveer SinghPooja HegdeJacqueline FernandezVa...,"T-Series Films, Reliance Entertainment, Rohit ...",[117]
103,D E C,23.0,Ganapath,Vikas Bahl,Tiger ShroffKriti Sanon,"Pooja Entertainment, Good Company Productions",[118]


In [44]:
df_2021 = df[['Title','Director', 'Cast']]

In [45]:
df_2021

,Title,Director,Cast
0,36 Farmhouse,Ram Ramesh Sharm,Vijay RaazSanjay MishraAmol ParasharBarkha Sin...
1,Hai Tujhe Salaam India,Avanish Kumar,Aarya BabbarAjaz KhanSmita GondkarKanwalpreet ...
2,Looop Lapeta,Aakash Bhatia,Taapsee PannuTahir Raj Bhasin
3,Gehraiyaan,Shakun Batra,Deepika PadukoneSiddhant ChaturvediAnanya Pand...
4,Badhaai Do,Harshavardhan Kulkarni,Rajkummar RaoBhumi Pednekar
...,...,...,...
100,No Means No,Vikash Verma,Dhruv VermaGulshan GroverSylwia CzechSharad Ka...
101,The Good Maharaja,Vikash Verma,Sanjay DuttDhruv VermaSharad KapoorGulshan Grover
102,Cirkus,Rohit Shetty,Ranveer SinghPooja HegdeJacqueline FernandezVa...
103,Ganapath,Vikas Bahl,Tiger ShroffKriti Sanon


In [46]:
!pip install tmdbv3api

Defaulting to user installation because normal site-packages is not writeable


In [47]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '71b41c0bf036bd0a8917fbc020e0a79d'

In [48]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [49]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_12272\3587185986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))


In [50]:
df_2021

,Title,Director,Cast,genres
0,36 Farmhouse,Ram Ramesh Sharm,Vijay RaazSanjay MishraAmol ParasharBarkha Sin...,Comedy Drama Mystery
1,Hai Tujhe Salaam India,Avanish Kumar,Aarya BabbarAjaz KhanSmita GondkarKanwalpreet ...,NaN
2,Looop Lapeta,Aakash Bhatia,Taapsee PannuTahir Raj Bhasin,Action Comedy Crime
3,Gehraiyaan,Shakun Batra,Deepika PadukoneSiddhant ChaturvediAnanya Pand...,Romance Drama
4,Badhaai Do,Harshavardhan Kulkarni,Rajkummar RaoBhumi Pednekar,Comedy Drama
...,...,...,...,...
100,No Means No,Vikash Verma,Dhruv VermaGulshan GroverSylwia CzechSharad Ka...,NaN
101,The Good Maharaja,Vikash Verma,Sanjay DuttDhruv VermaSharad KapoorGulshan Grover,NaN
102,Cirkus,Rohit Shetty,Ranveer SinghPooja HegdeJacqueline FernandezVa...,Comedy Drama
103,Ganapath,Vikas Bahl,Tiger ShroffKriti Sanon,Action Thriller


In [51]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [53]:
df_2021['director_name'] = df_2021['Cast'].map(lambda x: get_director(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_12272\3907705843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['director_name'] = df_2021['Cast'].map(lambda x: get_director(str(x)))


In [65]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(". ")[0])

In [66]:
df_2021['actor_1_name'] = df_2021['Cast'].map(lambda x: get_actor1(str(x)))

In [57]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [58]:
df_2021['actor_2_name'] = df_2021['Cast'].map(lambda x: get_actor2(str(x)))

In [59]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [60]:
df_2021['actor_3_name'] = df_2021['Cast'].map(lambda x: get_actor3(str(x)))

In [61]:
df_2021

,Title,Director,Cast,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,36 Farmhouse,Ram Ramesh Sharm,Vijay RaazSanjay MishraAmol ParasharBarkha Sin...,Comedy Drama Mystery,Vijay RaazSanjay MishraAmol ParasharBarkha Sin...,Vijay RaazSanjay MishraAmol ParasharBarkha Sin...,NaN,NaN
1,Hai Tujhe Salaam India,Avanish Kumar,Aarya BabbarAjaz KhanSmita GondkarKanwalpreet ...,NaN,Aarya BabbarAjaz KhanSmita GondkarKanwalpreet ...,Aarya BabbarAjaz KhanSmita GondkarKanwalpreet ...,NaN,NaN
2,Looop Lapeta,Aakash Bhatia,Taapsee PannuTahir Raj Bhasin,Action Comedy Crime,Taapsee PannuTahir Raj Bhasin,Taapsee PannuTahir Raj Bhasin,NaN,NaN
3,Gehraiyaan,Shakun Batra,Deepika PadukoneSiddhant ChaturvediAnanya Pand...,Romance Drama,Deepika PadukoneSiddhant ChaturvediAnanya Pand...,Deepika PadukoneSiddhant ChaturvediAnanya Pand...,NaN,NaN
4,Badhaai Do,Harshavardhan Kulkarni,Rajkummar RaoBhumi Pednekar,Comedy Drama,Rajkummar RaoBhumi Pednekar,Rajkummar RaoBhumi Pednekar,NaN,NaN
...,...,...,...,...,...,...,...,...
100,No Means No,Vikash Verma,Dhruv VermaGulshan GroverSylwia CzechSharad Ka...,NaN,Dhruv VermaGulshan GroverSylwia CzechSharad Ka...,Dhruv VermaGulshan GroverSylwia CzechSharad Ka...,NaN,NaN
101,The Good Maharaja,Vikash Verma,Sanjay DuttDhruv VermaSharad KapoorGulshan Grover,NaN,Sanjay DuttDhruv VermaSharad KapoorGulshan Grover,Sanjay DuttDhruv VermaSharad KapoorGulshan Grover,NaN,NaN
102,Cirkus,Rohit Shetty,Ranveer SinghPooja HegdeJacqueline FernandezVa...,Comedy Drama,Ranveer SinghPooja HegdeJacqueline FernandezVa...,Ranveer SinghPooja HegdeJacqueline FernandezVa...,NaN,NaN
103,Ganapath,Vikas Bahl,Tiger ShroffKriti Sanon,Action Thriller,Tiger ShroffKriti Sanon,Tiger ShroffKriti Sanon,NaN,NaN


In [62]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [63]:
new_df20 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [64]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Vijay RaazSanjay MishraAmol ParasharBarkha Sin...,Vijay RaazSanjay MishraAmol ParasharBarkha Sin...,NaN,NaN,Comedy Drama Mystery,36 Farmhouse
1,Aarya BabbarAjaz KhanSmita GondkarKanwalpreet ...,Aarya BabbarAjaz KhanSmita GondkarKanwalpreet ...,NaN,NaN,NaN,Hai Tujhe Salaam India
2,Taapsee PannuTahir Raj Bhasin,Taapsee PannuTahir Raj Bhasin,NaN,NaN,Action Comedy Crime,Looop Lapeta
3,Deepika PadukoneSiddhant ChaturvediAnanya Pand...,Deepika PadukoneSiddhant ChaturvediAnanya Pand...,NaN,NaN,Romance Drama,Gehraiyaan
4,Rajkummar RaoBhumi Pednekar,Rajkummar RaoBhumi Pednekar,NaN,NaN,Comedy Drama,Badhaai Do
...,...,...,...,...,...,...
100,Dhruv VermaGulshan GroverSylwia CzechSharad Ka...,Dhruv VermaGulshan GroverSylwia CzechSharad Ka...,NaN,NaN,NaN,No Means No
101,Sanjay DuttDhruv VermaSharad KapoorGulshan Grover,Sanjay DuttDhruv VermaSharad KapoorGulshan Grover,NaN,NaN,NaN,The Good Maharaja
102,Ranveer SinghPooja HegdeJacqueline FernandezVa...,Ranveer SinghPooja HegdeJacqueline FernandezVa...,NaN,NaN,Comedy Drama,Cirkus
103,Tiger ShroffKriti Sanon,Tiger ShroffKriti Sanon,NaN,NaN,Action Thriller,Ganapath


In [29]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [30]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      8
actor_3_name     26
genres            1
movie_title       1
comb             26
dtype: int64

In [31]:
new_df20 = new_df20.dropna(how='any')

In [32]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [33]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_10068\2267385682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['movie_title'] = new_df20['movie_title'].str.lower()


In [34]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,the dig,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,outside the wire,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
355,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
356,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
357,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
358,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [35]:
old_df = pd.read_csv('new4_data.csv')

In [36]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6278,Pete Docter,Jamie Foxx,Tina Fey,Graham Norton,Animation Family Comedy Fantasy Drama,soul,Jamie Foxx Tina Fey Graham Norton Pete Docter ...
6279,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
6280,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
6281,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [37]:
final_df = old_df.append(new_df20,ignore_index=True)

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_10068\3205022339.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df20,ignore_index=True)


In [38]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6613,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
6614,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
6615,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
6616,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [39]:
final_df.to_csv('new5_data.csv',index=False)